In [1]:
import pandas as pd
import numpy as np
import warnings
import time
import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
pd.set_option ('display.max_columns', None)
warnings.filterwarnings('ignore', category = FutureWarning)
warnings.filterwarnings('ignore', category = DeprecationWarning)

In [3]:
options = Options()

options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disabe-gpu")
options.add_argument("--no-sandbox")
#options.headless=True
#options.add_argument("--window-size=1400,1600")

## Past IGOs

In [5]:
titles = []
symbols= []
token_metrics = []
presale_info = []
ATH_ROI = []
ATH_value = []
basic_info = []

In [8]:
def open(link):
    try:
        time.sleep(5)
        browser.get(link)
    except:
        print('no internet connection')

In [7]:
def OpenProjectLink():
    
    open('https://igo.gamestarter.com/igo')
    
    time.sleep(2)
    
    browser.maximize_window()
    
    time.sleep(2)
    
    cookie = browser.find_element(by = 'xpath', value = '//div[@style = "cursor:pointer"]')
    
    try:
        cookie.click()
    except:
        pass
    
    time.sleep(2)
    
    i= 0
    
    while i <= 7:
        
        try:
            load = browser.find_element(by = 'xpath', value ='//div[@class = " font-bold "]')
            load.click()
            time.sleep(2)
            i +=1
        
        except:
            break
    
    find_length = len(browser.find_elements(by = 'xpath', value ='//a[@class = "absolute top-0 left-0 h-full w-full z-10"]'))
    
    list_count = 1

    while (list_count < find_length):
        
        global ptitles, psymbols, ptoken_metrics, pATH_ROI, pATH_value, ppresale_info, pbasic_info
        
        time.sleep(2)
        
        try:
            find = browser.find_elements(by = 'xpath', value ='//a[@class = "absolute top-0 left-0 h-full w-full z-10"]')
            
            find[list_count].click()
        
        except:
            pass
        
        time.sleep(3)
        
        try:
            title = browser.find_element(by ='xpath', value = '//div[@class = "h6 simple"]').text
            titles.append(title)
        
        except:
            titles.append('N/A')
        
        try:
            symbol = browser.find_element(by ='xpath', value = '//div[@class = "font-semibold text-base"]').text
            symbols.append(symbol)
            
        except:
            symbols.append('N/A') 
            
        try:
            presale = browser.find_element(by ='xpath', value = '//div[@class = "dark:text-dark-text font-semibold"]')
            presale_info.append(presale.text)
            
        except:
            presale_info.append('N/A')
        
        time.sleep(2)
            
        try:
            basic = browser.find_elements(by ='xpath', value = '//div[@class = "igo-txt dark:font-normal"] //p//ul')
            basic_info.append(basic.text)
            
        except:
            basic_info.append('N/A')
            
        try:
            metrics = browser.find_elements(by ='xpath', value = '//table[@width ="100%"]')
            for metric in metrics:
                token_metrics.append(metric.text)
            
        except:
            token_metrics.append('N/A')
                    
        time.sleep(2)

        #go back
        browser.back()
        time.sleep(2)
        if list_count == find_length-1:
            
            try:
                ATH = browser.find_elements(by ='xpath', value = '//div[@class = "px-2 text-sm text-white font-bold"]')
                for val in ATH:
                    ATH_ROI.append(val.text)   
            except:
                ATH_ROI.append('N/A')
                
            try:
                ATH = browser.find_elements(by ='xpath', value = '//div[@class = "text-black dark:text-white font-bold text-xl"]')
                for val in ATH:
                    ATH_value.append(val.text)
            except:
                ATH_value.append('N/A')
                
            break
        list_count +=1
        
    return "AWESOME"

In [8]:
browser = webdriver.Chrome(executable_path="C:\webdrivers\chromedriver.exe", options = options)
OpenProjectLink()

'AWESOME'

In [9]:
#working on symbols

tickers = [symbol.replace('/',' ').replace('BUSD',' ').strip() for symbol in symbols]

In [10]:
#working on token metrics

values = [value.replace('\n',' ') for value in token_metrics]
details = [re.sub(r'Private.+','',item) for item in values]
platform_details = [i for i in details if i]

metrics = [item.split(':') for item in platform_details]

raise_amount = []
price = []
lock_up = []
for i in range(1, len(metrics)+1):
    lock_up.append(metrics[-i][-1].strip())
    price.append(metrics[-i][-2])
    raise_amount.append(metrics[-i][-3])

lock_up = lock_up[::-1]
token_price = [re.sub(r'[a-zA-Z].+','',item).strip() for item in price]
token_price = token_price[::-1]
platform_raise = [re.sub(r'[a-zA-Z].+','',item).strip() for item in raise_amount]
platform_raise = platform_raise[::-1]
platform_raise[1] = '$125,000'

In [11]:
#working on presale info

sale_info = [word for line in presale_info for word in line.split('\n')]

price_type = [sale_info[i] for i in range(0,len(sale_info),4)]
sale_period = [sale_info[i] for i in range(1,len(sale_info),4)]
FCFS_period = [sale_info[i] for i in range(2,len(sale_info),4)]
allocation = [sale_info[i] for i in range(3,len(sale_info),4)]

presale_period = [re.sub(r'Sale\s','',item) for item in sale_period]
FCFS = [re.sub(r'FCFS\s','',item) for item in FCFS_period]
base_allocation = [re.sub(r'.*(.+:)\s','',item) for item in allocation]

In [ ]:
# Final dataframe

dataframe_list = [titles, tickers, token_price, lock_up, ATH_value, ATH_ROI, platform_raise, presale_period, FCFS, 
                  base_allocation]
labels= ['Name','Symbol','Price','Vesting_period','ATH_value','ATH_ROI','Platform_raise','Presale_period','FCFS_period',
         'Base_allocation']
dataframe = pd.DataFrame(np.column_stack(dataframe_list), columns = labels)

dataframe.head()

In [ ]:

    presale = browser.find_element(by = 'xpath', value = '//div[@class = "w-full flex lg:flex-row flex-col items-center"]//a')
    time.sleep(2)
    presale.click()
    time.sleep(8)
    
    title = browser.find_element(by ='xpath', value = '//div[@class = "h6 simple"]').text
    titles.append(title)
    
    symbol = browser.find_element(by ='xpath', value = '//div[@class = "font-semibold text-base"]').text
    symbols.append(symbol)

    req = browser.find_element(by = 'xpath', value = '//p[@class = "text-lg font-normal"]')
    requirements.append(req.text)
    
    metrics = browser.find_elements(by ='xpath', value = '//table[@width ="100%"]')
    for metric in metrics :
        token_metrics.append(metric.text)
    time.sleep(2)
    
    allocation = browser.find_element(by = 'xpath', value = '//div[@class = "flex justify-between items-center font-normal"]')
    pools.append(allocation.text)
    
    countdown = browser.find_element(by = 'xpath', value = '//p[@class = "text-block-190 font-semibold"]')
    timer.append(countdown.text)
    
    presale = browser.find_element(by = 'xpath', value = '//div[@class = "dark:text-dark-text font-semibold"]')
    presale_info.append(presale.text)

### Incoming and Next IGOs

In [84]:
sales_liquid = '//div[@class="flex justify-between items-center font-normal"]//div' #0:BUSD, 1:amount
socials = '//div[@class="flex items-center space-x-6 space-x-8 py-6"]//a'
state = '//div[@class="flex"]'
#state = '//p[@class="text-block-190 font-semibold"]'
presale = '//div[@class="dark:text-dark-text font-semibold"]//div'
#p_info = '//div[@class = "dark:text-dark-text font-semibold"]//div'

In [85]:
titles = []
symbols= []
token_metrics = []
base_allocation = []
registration = []
sales = []
requirements = []
pools = []
pool_worth = []
open_pool = []
website = []
twitter = []
telegram = []
discord = []
status = []

In [87]:
def OpenProjectLink():
    time.sleep(5)
    
    open('https://igo.gamestarter.com/')
    
    time.sleep(5)
    
    browser.maximize_window()
    
    time.sleep(5)
    
    cookie = browser.find_element(by = 'xpath', value = '//div[@style = "cursor:pointer"]')
    
    try:
        cookie.click()
    except:
        pass
    
    #timer = browser.find_element(by = 'xpath', value = '//div[@class = "flex flex-row space-x-2"]').text
    global titles, symbols, requirements, token_metrics, sales, registration, pools, pool_worth, base_allocation, open_pool, website, telegram, twitter, discord, status

    try:
        stats = browser.find_elements(by = 'xpath', value = state)
        for val in stats:
            status.append(val.text)
    except:
        status.append('TBA')

    time.sleep(3)

    find_length = len(browser.find_elements(by = 'xpath', value ='//div[@class="flex lg:flex-row flex-col items-center"]'))
    
    list_count = 0

    while (list_count < find_length):
        
        try:
            find = browser.find_elements(by = 'xpath', value ='//div[@class="flex lg:flex-row flex-col items-center"]')
            find[list_count].click()
        except:
            try:
                find[list_count+2].click()
            except:
                pass
        time.sleep(3)
        
        try:
            title = browser.find_elements(by ='xpath', value = '//div[@class = "h6 simple"]')[0].text
            titles.append(title)
        
        except:
            titles.append('N/A')
        
        try:
            symbol = browser.find_element(by ='xpath', value = '//div[@class = "font-semibold text-base"]').text
            symbols.append(symbol)
            
        except:
            symbols.append('N/A')
            
        try:
            req = browser.find_element(by = 'xpath', value = '//p[@class = "text-lg font-normal"]')
            requirements.append(req.text)
        
        except:
            requirements.append('TBA')
            
        try:
            metrics = browser.find_elements(by ='xpath', value = '//table[@width ="100%"]')
            for metric in metrics :
                token_metrics.append(metric.text)
            
        except:
            token_metrics.append('TBA')
        
        try: 
            liquidity = browser.find_elements(by = 'xpath', value = sales_liquid)[1]
            pools.append(liquidity.text)
            
        except:
            pools.append('TBA')
        
        try: 
            worth = browser.find_elements(by = 'xpath', value = sales_liquid)[0]
            pool_worth.append(worth.text)
            
        except:
            pool_worth.append('TBA')

        try:
            reg = browser.find_elements(by = 'xpath', value = presale)[1]
            registration.append(reg.text)
            
        except:
            registration.append('TBA')

        try:
            sale = browser.find_elements(by = 'xpath', value = presale)[2]
            sales.append(sale.text)
            
        except:
            sales.append('TBA')
        
        try: 
            open_sale = browser.find_elements(by = 'xpath', value = presale)[3]
            open_pool.append(open_sale.text)
            
        except:
            open_pool.append('TBA')
        
        try:
            allocation = browser.find_elements(by = 'xpath', value = presale)[-1]
            base_allocation.append(allocation.text)
            
        except:
            base_allocation.append('TBA')

        try:
            web_link = browser.find_elements(by ='xpath', value = socials)[2]
            website.append(web_link.get_attribute("href"))
        except:
            website.append('Not Available')

        try:
            telegram_link = browser.find_elements(by ='xpath', value = socials)[1]
            telegram.append(telegram_link.get_attribute("href"))
        except:
            telegram.append('Not Available')

        try:
            twitter_link = browser.find_elements(by ='xpath', value = socials)[0]
            twitter.append(twitter_link.get_attribute("href"))
        except:
            twitter.append('Not Available')
        
        try:
            discord_link = browser.find_elements(by ='xpath', value = socials)[3]
            discord.append(web_link.get_attribute("href"))
        except:
            discord.append('Not Available')
        
                                            
        time.sleep(3)
        #go back
        browser.back()
        time.sleep(10)
        
        if list_count == find_length-1:
            break
        list_count +=1
        
    return "cool"

In [88]:
browser = webdriver.Chrome(executable_path="C:\webdrivers\chromedriver.exe", options = options)
OpenProjectLink()

'cool'

In [ ]:
requirements = [val.replace("This IDO requires","").replace('.','').strip() for val in requirements]


In [ ]:
pools = [pool.replace("0 / ","").strip() for pool in pools]

In [92]:
# working on symbols, titles and status

project_name = [ title for title in titles if "N/A" not in title]
symbols = [symbol.replace("N/A","Not Found").replace('/',' ').replace('BUSD',' ').strip() for symbol in symbols]
tickers = [ ticker for ticker in symbols if "Not Found" not in ticker]

tickers = tickers[:-2]
project_name = project_name[:-2]
for i in range(0,len(project_name) - len(status)):
    status.append('TBA')


In [94]:
website = website[:-2]
twitter = twitter[:-2]
telegram = telegram[:-2]
discord = discord[:-2]
requirements = requirements[:-2]
pools = pools[:-2]
open_pool = open_pool[:-2]

In [95]:
len(website), len(requirements), len(pools), len(open_pool), len(status)

(12, 12, 12, 12, 12)

In [96]:
#working on token metrics

values = [value.replace('\n',' ') for value in token_metrics]
details = [re.sub(r'Private.+','',item) for item in values]
platform_details = [i for i in details if i]

metrics = [item.split(':') for item in platform_details]

raise_amount = []
price = []
lock_up = []
for i in range(1, len(metrics)+1):
    lock_up.append(metrics[-i][-1].strip())
    price.append(metrics[-i][-2])
    raise_amount.append(metrics[-i][-3])

lock_up = lock_up[::-1]
token_price = [re.sub(r'[a-zA-Z].+','',item).strip() for item in price]
token_price = token_price[::-1]
platform_raise = [re.sub(r'[a-zA-Z].+','',item).strip() for item in raise_amount]
platform_raise = platform_raise[::-1]

In [77]:
for i in range(0,len(project_name) - len(token_price)):
    token_price.append('TBA')
sales_price = token_price[:-1]
len(sales_price)

11

In [104]:
registration = registration[:-2]
base_allocation = base_allocation[:-2]
sales = sales[:-2]

In [ ]:
status = [stat.replace("\n","").replace(":"," : ") for stat in status]

In [148]:
dataframe_list = [project_name, tickers, token_price, lock_up, platform_raise, registration, sales, status, requirements, pools, pool_worth, website,
                 twitter, telegram]
labels= ['Project','symbol','price','vesting_period','platform_raise','registration_period', 'sales_period', 'Presale_status','sales_requirements','sales_supply', 'sales_value',
        'website', 'twitter', 'telegram']

#base_allocation, FCFS, discord

In [149]:
dataframe = pd.DataFrame(np.column_stack(dataframe_list), columns = labels)
dataframe = dataframe.drop(10).reset_index(drop=True)
dataframe.head()

,Project,symbol,price,vesting_period,platform_raise,registration_period,sales_period,Presale_status,sales_requirements,sales_supply,sales_value,website,twitter,telegram
0,WALKN,WALKN,$0.0049,"10% at TGE, the rest Linear Vesting over 6 months","$50,000","Registration 10 Jun, 11:00 – 13 Jun, 11:00 UTC","Sale 13 Jun, 14:00 – 16 Jun, 14:00 UTC",Starts in : 00d6h33m,KYC verification,"10,204,080 WALKN",0 BUSD,https://walkn.app/,https://twitter.com/WalkN_app,https://t.me/walknapp
1,REVOLAND,REVO,$0.6,100% at TGE,"$50,000","Registration 14 Jun, 11:00 – 17 Jun, 11:00 UTC","Sale 17 Jun, 14:00 – 20 Jun, 14:00 UTC",Registration in : 1d3h33m,KYC verification,"83,335 REVO",0 BUSD,https://www.revoland.com/,https://twitter.com/revolutionland,https://t.me/revolandofficial
2,KOLNET,KOL,$0.012,"10% at TGE, then 30% monthly","$100,000","Registration 23 Jun, 11:00 – 26 Jun, 11:00 UTC","Sale 27 Jun, 14:00 – 28 Jun, 14:00 UTC",Registration in : 10d3h33m,KYC verification,"4,166,650 KOL",0 BUSD,https://kolnet.io/,https://twitter.com/kolnet_official,https://t.me/KOLnet_Official
3,KOAKUMA,KKMA,$0.03,"25% TGE, 1 month cliff, vesting over 3 months","$150,000",Registration TBA,Sale TBA,TBA,KYC verification,"4,999,950 KKMA",0 BUSD,https://koakuma.io/,https://twitter.com/koakuma_game,https://t.me/koakuma_official
4,CHIBI DINOS,HOOP,$0.055,"20% on TGE, 6 month linear vesting","$250,000",Registration TBA,Sale TBA,TBA,KYC verification,"4,545,450 HOOP",0 BUSD,https://discord.com/invite/dinos,https://twitter.com/DinosChibi,https://www.chibidinos.io/


In [151]:
dataframe.to_csv("Gamestarter_live_upcoming.csv", index=False)
#dataframe = pd.read_csv("GameFi_upcoming_launchpad.csv")
#new_df = pd.concat([dataframe, details]).drop_duplicates()
#new_df.to_csv("GameFi_upcoming_launchpad.csv", index=False)